In [30]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from connectors.data_collector import DataCollector, mt5

import pandas as pd 
import numpy as np

from IPython.display import display 


In [32]:
data_col = DataCollector()
data = data_col.get_historical_data(symbol="EURUSD", timeframe= mt5.TIMEFRAME_M5 )
data.head() 

,Date,Open,Close,High,Low,Volume
0,2024-03-05 03:45:00,1.08513,1.08491,1.08513,1.08484,266
1,2024-03-05 03:50:00,1.08489,1.08488,1.08491,1.08472,236
2,2024-03-05 03:55:00,1.08489,1.08493,1.08501,1.08484,196
3,2024-03-05 04:00:00,1.08493,1.08496,1.08502,1.08484,366
4,2024-03-05 04:05:00,1.08495,1.08510,1.08512,1.08493,564


In [36]:
def create_lag_features(data:pd.DataFrame, start:int=1, end:int|None=None, column:str="Close"):
    """
    Creates lagged features for the specified column of the time series.

    Parameters:
    data (pd.DataFrame): The original DataFrame containing the time series.
    start (int): The starting lag (default 1).
    end (int | None): The ending lag. If None, calculated as half the square root of the series length.
    column (str): The name of the column for which to create lagged features (default "Close").

    Returns:
    pd.DataFrame: The DataFrame with the lagged features added.
    """

    df = data.copy()

    if end is None:
        n = len(data)
        end = int((n / np.sqrt(n)) / 2)

    for lag in range(start, end ):
        df[f'lag_{lag}'] = df[column].shift(lag)

    return df.dropna()


# lag_data = create_lag_features(data)
# lag_data.head()

In [37]:
data_col